# Spelling Bee Challenge with RNN

- Uses seq2seq model

[reference](https://github.com/mikesj-public/rnn_spelling_bee/blob/master/spelling_bee_RNN.ipynb)

## Preprocessing

**what do we need?**

- Dataset : Training and test set (x's and y's)
- Phonemes : index to phoneme and phoneme to index dictionaries
- Alphabets : index to alphabets and alphabets to index dictionaries 

In [1]:
START_LINE = 126
END_LINE = 133905
FILENAME = 'data/cmudict-0.7b'
lines = open(FILENAME, 'r', encoding='utf-8', errors='ignore').read().split('\n')[START_LINE:END_LINE]

In [2]:
import random
random.choice(lines)

'DAMERLIN(1)  D AE1 M ER0 L IH0 N'

In [3]:
phonemes = []
words = []

for line in lines:
    word, pronounce = line.split('  ')
    phonemes.append(pronounce.split(' '))
    words.append(word)

In [4]:
print(random.choice(words))
print(random.choice(phonemes))

STITZEL
['K', 'AH0', 'D', 'AA1', 'F', 'IY0']


In [5]:
phoneme_vocab = set([item for row in phonemes for item in row])
phoneme_vocab = ['_'] + sorted(list(phoneme_vocab))

In [6]:
phoneme_vocab_size = len(phoneme_vocab)

In [7]:
# need idx2phoneme and phoneme2idx
idx2phoneme = dict(enumerate(phoneme_vocab))
phoneme2idx = dict(zip(idx2phoneme.values(), idx2phoneme.keys()))

In [8]:
print(idx2phoneme[34])
print(phoneme2idx['N'])

HH
45


In [9]:
# aphabets
idx2alpha = dict(enumerate('_abcdefghijklmnopqrstuvwxyz'))
alpha2idx = dict(zip(idx2alpha.values(), idx2alpha.keys()))
alpha_vocab_len = len('_abcdefghijklmnopqrstuvwxyz')

In [10]:
# - word to phoneme_index dictionaries
# - remember words and phonemes? we are gonna use them here
# - lets also remove too small (<5 characters) or too large (>15 characters) words
word2phoneme_idx = {}
for word, phoneme_list in zip(words, phonemes):
    if len(word) > 5 and len(word) < 15 and len(phoneme_list) < 16:
        word2phoneme_idx[word.lower()] = [phoneme2idx[phoneme] for phoneme in phoneme_list]

### Typically the final stage of preprocessing
- convert dataset to numpy arrays filled with indices instead of characters, with padding
- split into training, validation, test sets

In [ ]:
import numpy as np

updated_words = word2phoneme_idx.keys()
dataset_len = len(updated_words)

# empty numpy arrays to hold the indices
dataX = np.zeros([dataset_len,16])
dataX = np.zeros([dataset_len,15])

for i,word in enumerate(updated_words):
    phoneme_list = word2phoneme_idx[word]
    # add items to dataX and dataY with padding
    for j, n in 
    

In [11]:
import numpy as np

pairs = np.random.permutation(list(word2phoneme_idx.keys()))

input_ = np.zeros((len(pairs), 16))
labels_ = np.zeros((len(pairs), 15))

for i, k in enumerate(pairs):
    v = word2phoneme_idx[k]
    k = k + "_" * (15 - len(k))
    v = v + [0] * (16 - len(v))
    
    for j, n in enumerate(v):
        input_[i][j] = n
    for j, letter in enumerate(k):
        if letter in alpha2idx:
            labels_[i][j] = alpha2idx[letter]
        
input_ = input_.astype(np.int32)
labels_ = labels_.astype(np.int32)

input_test   = input_[:10000]
input_val    = input_[10000:20000]
input_train  = input_[20000:]
labels_test  = labels_[:10000]
labels_val   = labels_[10000:20000]
labels_train = labels_[20000:]

data_test  = zip(input_test, labels_test)
data_val   = zip(input_val, labels_val)
data_train = zip(input_train, labels_train)

In [13]:
print(input_test[124], labels_test[124])

[68  7 45 24 43  7  0  0  0  0  0  0  0  0  0  0] [26  1 14  5 12 12  1  0  0  0  0  0  0  0  0]


## Moving on to Tensorflow

In [31]:
import tensorflow as tf

In [48]:
xseq_len = 16
yseq_len = 15
batch_size = 128
xvocab_size = 70
yvocab_size = 28
emb_dim = 128

In [64]:
tf.reset_default_graph()

In [65]:
x_ = [ tf.placeholder(tf.int32, shape=[None,], name='x{}'.format(i)) for i in range(xseq_len)]
y_ = [ tf.placeholder(tf.int32, shape=[None,], name='y{}'.format(i)) for i in range(yseq_len)]
decoder_inputs = [tf.zeros_like(x_[0], dtype=tf.int32, name = "GO")] + y_[:-1]
keep_prob = tf.placeholder(tf.float32)
basic_cell = tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(emb_dim),
        output_keep_prob=keep_prob)
stacked_lstm = tf.nn.rnn_cell.MultiRNNCell([basic_cell]*3)


with tf.variable_scope('decoder') as scope:
    decode_outputs, decode_states = tf.nn.seq2seq.embedding_rnn_seq2seq(x_,decoder_inputs, stacked_lstm,
                                        xvocab_size, yvocab_size, emb_dim)
    scope.reuse_variables()
    # testing
    decode_outputs_test, decode_states_test = tf.nn.seq2seq.embedding_rnn_seq2seq(
        x_, decoder_inputs, stacked_lstm, xvocab_size, yvocab_size,emb_dim,
        feed_previous=True)

In [66]:
# we weight the losses based on timestep of decoder output
loss_weights = [tf.ones_like(l, dtype=tf.float32) for l in y_] # gives [1, 1, ..., 1,1] - equal weights
loss = tf.nn.seq2seq.sequence_loss(decode_outputs, y_, loss_weights, yvocab_size)
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)